# Weakly Connected Components (WCC)

In this notebook, we will use cuGraph to compute the weakly connected components and attach a component label to each vertex.

Notebook Credits
* Original Authors: Kumar Aatish
* Last Edit: 08/02/2019

RAPIDS Versions: 0.9.0    

Test Hardware

* GV100 32G, CUDA 10,1



## Introduction

~Single source shortest path computes the shortest paths from the given starting vertex to all other reachable vertices.~ 

To compute WCC for a graph in cuGraph we use:
**cugraph.weakly_connected_components(G)**

Input
* __G__: cugraph.Graph object

Returns 
* __df__:  a cudf.DataFrame object with one column:
    * df['labels']: The label identifier for the vertex


## cuGraph 0.7 Notice 
cuGraph version 0.7 has some limitations:
* Only Int32 Vertex ID are supported
* Only float (FP32) edge data is supported
* Vertex numbering is assumed to start at zero

These limitations are being addressed and will be fixed future versions.  
These example notebooks will illustrate how to manipulate the data so that it comforms to the current limitations    
    



### Test Data
We will be using the Zachary Karate club dataset 
*W. W. Zachary, An information flow model for conflict and fission in small groups, Journal of
Anthropological Research 33, 452-473 (1977).*


![Karate Club](./img/zachary_black_lines.png)

This is a small graph which allows for easy visual inspection to validate results.  
__Note__: The Karate dataset starts with vertex ID 1 which the cuGraph analytics assume a zero-based starting ID.  

In [1]:
# Import needed libraries
import cugraph
import cudf
import numpy as np
from collections import OrderedDict

### Read the data and adjust the vertex IDs

In [2]:
# Test file  - using the clasic Netscience club dataset.  
datafile='./data/netscience.csv'

In [3]:
# Read the data file
cols = ["src", "dst", "wgt"]

dtypes = OrderedDict([
        ("src", "int32"), 
        ("dst", "int32"),
        ("wgt", "float32")
        ])
usecols = ["src", "dst"]
gdf = cudf.read_csv(datafile, names=cols, usecols = usecols, delimiter=' ', dtype=list(dtypes.values()) )

In [4]:
gdf.head().to_pandas()

,src,dst
0,1,0
1,1084,0
2,946,1
3,1084,1
4,3,2


### Create a Graph and call Weakly Connected Components

In [5]:
# create a Graph 
G = cugraph.Graph()
G.add_edge_list(gdf["src"], gdf["dst"])

In [11]:
# Call cugraph.weakly_connected_components:
df = cugraph.weakly_connected_components(G)
print(df)

    labels
0       1
1       1
2       3
3       3
4       3
5       3
6       3
7       8
8       8
9       8
[1579 more rows]


In [7]:
df['vertex_id'] = df.index
label_gby = df.groupby('labels')
label_count = label_gby.count()
largest_component = label_count.nlargest(n = 1, columns = 'vertex_id')

In [8]:
print("Total number of components found : ", len(label_count))

Total number of components found :  396


In [9]:
print("Size of the largest component is found to be : ", largest_component['vertex_id'][0])

Size of the largest component is found to be :  379


In [10]:
expr = "labels == 1"
component = df.query(expr)
print("Vertex Ids that belong to component label 1 : ")
print(component)

Vertex Ids that belong to component label 1 : 
   labels  vertex_id
0       1          0
1       1          1
946       1        946
1084       1       1084


___
Copyright (c) 2019, NVIDIA CORPORATION.

Licensed under the Apache License, Version 2.0 (the "License");  you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.
___

In [ ]:
small_comp_id = df.query()